# Iterative Backtesting - Part 5 | Test Strategy

In [6]:
import pandas as pd
import datetime
from datetime import timedelta
import numpy as np

In [7]:
class Backtester():
    def __init__(self, ticker, ema_short_period, ema_long_period, amount):
        self.symbol = ticker
        self.ema_s = ema_short_period
        self.ema_l = ema_long_period
        self.result = None
        self.trades = 0
        self.units = 0
        self.logs = pd.DataFrame()
        self.data = pd.DataFrame()
        self.position = 0
        self.initial_balance = amount
        self.current_balance = amount

    def reset_variables(self):
        self.result = None
        self.trades = 0
        self.units = 0
        self.logs = pd.DataFrame()
        self.data = pd.DataFrame()
        self.position = 0
        self.current_balance = self.initial_balance
        
    def set_params(ema_short_period, ema_long_period):
        self.ema_s = ema_short_period
        self.ema_l = ema_long_period
        
    def fetch_data(self):
        # API, Database, csv file
        data = pd.read_csv('eurusd.csv')
        data['date'] = pd.to_datetime(data['date'])
        data.set_index('date', drop=True, inplace=True)
        data['return'] = np.log(data.close/data.close.shift(1))
        data['ema_short'] = data['close'].ewm(span=self.ema_s, adjust=False).mean()
        data['ema_long'] = data['close'].ewm(span=self.ema_l, adjust=False).mean()
        self.data = data
    def plot_price(self):
        self.data[['close', 'ema_short', 'ema_long']].plot(figsize=(14, 7))

    def access_data(self, bar):
        date_value = self.data.index[bar].date()
        price = float(self.data['close'].iloc[bar])
        ema_short = float(self.data['ema_short'].iloc[bar])
        ema_long = float(self.data['ema_long'].iloc[bar])
        return date_value, price, ema_short, ema_long
        
    def plot_return(self):
        self.data['return'].plot(figsize=(14, 7))

    def get_balance(self):
        return self.current_balance
    def get_current_position(self, bar):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        current_position = self.units * price
        print(f" Current Position Value: {current_position}")

    def get_current_NAV(self, bar):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        current_position = self.units * price
        current_NAV = self.current_balance + current_position
        print(f" Current NAV: {current_NAV}")
        
    def buy_instrument(self, bar, qty):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        self.units = self.units + qty
        self.current_balance = self.current_balance - (price * qty)
        self.trades = self.trades + 1
        print(f"{date_value} | BUY {qty} @ $: {round(price, 2) }")
    def sell_instrument(self, bar, qty):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        self.units = self.units - qty
        self.current_balance = self.current_balance + (price * qty)
        self.trades = self.trades + 1        
        print(f"{date_value} | SELL {qty} @ $: {round(price, 2) }")
        
    def close_positions(self, bar):
        date_value, price, ema_short, ema_long = self.access_data(bar)
        if self.position != 0:
            print("\n")
            print('$$$ - Closing positions Finally - $$$')
            if self.position == 1:
                self.sell_instrument(bar = bar, qty = abs(self.units))
            elif self.position == -1:
                self.buy_instrument(bar = bar, qty = abs(self.units))
            self.position = 0
        performance = ((self.current_balance - self.initial_balance) / self.initial_balance) * 100
        print("\n")
        print("------------------- Summary -----------------")
        print(f"Performance: {round(performance, 2)} %")
        self.net_balance = self.get_balance()
        print(f"Current Balance: {self.net_balance}")
        print(f"Number of Trades: {self.trades}")
        
    def test_strategy_one(self):
        self.reset_variables()
        self.fetch_data()
        qty = 50000
    
        for bar in range(len(self.data) - 1):
            close = self.data['close'].iloc[bar]
            ema_s = self.data['ema_short'].iloc[bar]
            ema_l = self.data['ema_long'].iloc[bar]
    
            previous_position = self.position  # store old state
    
            if ema_s > ema_l:  # bullish
                if self.position == 0:
                    self.buy_instrument(bar=bar, qty=qty)
                    self.position = 1
                elif self.position == -1:
                    self.buy_instrument(bar=bar, qty=qty * 2)
                    self.position = 1
    
            elif ema_l > ema_s:  # bearish
                if self.position == 0:
                    self.sell_instrument(bar=bar, qty=qty)
                    self.position = -1
                elif self.position == 1:
                    self.sell_instrument(bar=bar, qty=qty * 2)
                    self.position = -1
    
            # Check if position actually changed
            if previous_position != self.position:
                balance = self.get_balance()
                print(f"Current Balance: {balance}")
    
        self.close_positions(-1)


In [8]:
obj = Backtester(ticker = 'NIFTY', ema_short_period = 50, ema_long_period = 200, amount = 70000)

In [9]:
obj.symbol, obj.ema_s, obj.ema_l, obj.trades

('NIFTY', 50, 200, 0)

In [10]:
obj.test_strategy_one()

2004-01-05 | BUY 50000 @ $: 1.27
Current Balance: 6565.124988555908
2004-02-02 | SELL 100000 @ $: 1.24
Current Balance: 130895.28799057007
2004-02-09 | BUY 100000 @ $: 1.27
Current Balance: 3954.6799659729004
2004-03-03 | SELL 100000 @ $: 1.22
Current Balance: 125864.27450180054
2004-10-18 | BUY 100000 @ $: 1.25
Current Balance: 973.5536575317383
2005-05-26 | SELL 100000 @ $: 1.25
Current Balance: 126112.76626586914
2006-04-25 | BUY 100000 @ $: 1.24
Current Balance: 1822.7767944335646
2008-09-16 | SELL 100000 @ $: 1.41
Current Balance: 143173.52294921872
2009-06-08 | BUY 100000 @ $: 1.39
Current Balance: 4143.67437362668
2010-02-05 | SELL 100000 @ $: 1.37
Current Balance: 140944.45228576657
2010-10-13 | BUY 100000 @ $: 1.39
Current Balance: 1775.2957344054885
2011-01-06 | SELL 100000 @ $: 1.32
Current Balance: 133343.85871887204
2011-01-26 | BUY 100000 @ $: 1.37
Current Balance: -3516.833782196074
2011-09-28 | SELL 100000 @ $: 1.36
Current Balance: 132306.59484863278
2012-12-07 | BUY 1